<a href="https://colab.research.google.com/github/chandcalnaido/ColabDev/blob/main/GlassOD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#!pip install scipy

In [ ]:
!pip install pyod
!pip install suod

In [61]:
import pandas as pd
import numpy as np
from numpy import genfromtxt
import csv
from scipy.io import arff
from io import StringIO
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import seaborn as sns
from pyod.models.iforest import IForest
from pyod.models.pca import PCA
from pyod.models.knn import KNN
from pyod.models.copod import COPOD
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.suod import SUOD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import label_binarize
from pyod.models.suod import SUOD
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.copod import COPOD
from pyod.utils.utility import standardizer
from pyod.utils.data import generate_data
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize

#from sklearn.datasets import fetch_openml

In [96]:
data, meta  = arff.loadarff('/content/drive/MyDrive/ML/Data Sets/Glass_withoutdupl_norm.arff')


In [97]:
data = pd.DataFrame(data)
data = data.drop('id', axis=1)
#data = data.drop('outlier', axis=1)
data.head()

,var_0000,var_0001,var_0002,var_0003,var_0004,var_0005,var_0006,outlier
0,0.467651,0.321584,0.768880,0.246630,0.838799,0.099737,0.298340,b'no'
1,0.496412,0.220491,0.776032,0.316598,0.919973,0.089145,0.279479,b'no'
2,0.519133,0.404464,0.768012,0.334978,0.801622,0.092369,0.271238,b'no'
3,0.199650,0.547373,0.374284,0.362223,0.817017,0.000000,0.177913,b'yes'
4,0.847261,0.286361,0.000000,0.217792,0.000000,0.019135,1.000000,b'no'


In [98]:
X = data.iloc[:,0:7]
display(X)

,var_0000,var_0001,var_0002,var_0003,var_0004,var_0005,var_0006
0,0.467651,0.321584,0.768880,0.246630,0.838799,0.099737,0.298340
1,0.496412,0.220491,0.776032,0.316598,0.919973,0.089145,0.279479
2,0.519133,0.404464,0.768012,0.334978,0.801622,0.092369,0.271238
3,0.199650,0.547373,0.374284,0.362223,0.817017,0.000000,0.177913
4,0.847261,0.286361,0.000000,0.217792,0.000000,0.019135,1.000000
...,...,...,...,...,...,...,...
209,0.495555,0.438735,0.749682,0.277775,0.791251,0.092286,0.260617
210,0.515273,0.346053,0.777602,0.548135,0.863624,0.108131,0.230353
211,0.459787,0.517479,0.773646,0.285545,0.761141,0.057688,0.223982
212,0.442959,0.297568,0.631711,0.421653,0.884864,0.113589,0.276465


In [99]:
y = data.iloc[:,7]
#display(y)

# Convert the target variable to a NumPy array
y = np.array(y)

# Replace all occurrences of 'no' with 0 and 'yes' with 1
y[y == b'no'] = 0
y[y == b'yes']= 1



In [102]:
display(y)


array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object)

dtype('O')

In [104]:
lb = LabelBinarizer()

y = y.astype(int)

# For your array y, which is already binary:
y_transformed = lb.fit_transform(y)

print(y_transformed)

[[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]


In [105]:
print(y)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [107]:
display(y_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

In [108]:
# train SUOD
clf_name = 'SUOD'

# initialized a group of outlier detectors for acceleration
detector_list = [LOF(n_neighbors=15), LOF(n_neighbors=20),
                  LOF(n_neighbors=25), LOF(n_neighbors=35),
                  COPOD(), IForest(n_estimators=100),
                  IForest(n_estimators=200)]

# decide the number of parallel process, and the combination method
#clf = SUOD(base_estimators=detector_list, n_jobs=-1, combination='average',
#           verbose=False)

# or to use the default detectors
clf = SUOD(n_jobs=2, combination='average',
            verbose=False)
clf.fit(X_train)

RandomForestRegressor()



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    5.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.3s finished


SUOD(approx_clf=None, approx_clf_list=None, approx_flag_global=True,
   approx_ng_clf_list=None,
   base_estimators=[LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=15, novelty=True, p=2), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=20, novelty=True, p=2...features=1.0,
    max_samples='auto', n_estimators=150, n_jobs=1, random_state=None,
    verbose=0)],
   bps_flag=True, combination='average', contamination=0.1,
   jl_method='basic', n_jobs=2, rp_clf_list=None, rp_flag_global=True,
   rp_ng_clf_list=None, target_dim_frac=0.5, verbose=False)

In [109]:
display(clf.labels_)
display(clf.decision_scores_)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

array([-0.94174353,  0.07290135,  0.48441584,  0.29108766, -0.05418045,
        0.4918471 , -0.6654036 , -0.74496763,  0.61358129,  2.53069441,
        0.70989566, -0.9233734 , -0.3190996 , -0.14411165, -0.04845913,
        1.48649389,  0.39497758, -0.13156485, -0.58615818,  2.47893094,
       -0.74449971,  1.07244829, -0.79663386,  0.19702066, -0.375034  ,
       -0.58571858, -0.35706745,  0.0834915 , -0.46484618, -0.90205368,
       -0.82423298,  0.63375771, -0.58896131, -0.61596971, -0.75299896,
       -0.57365835,  0.45101996, -0.67739264, -0.52267543,  1.69711294,
       -0.88246704,  2.57736525, -0.22244196, -0.20274159, -0.22125057,
        0.32045979, -0.83039248, -0.77936739,  2.84614232, -0.6348763 ,
        1.10383489,  0.00798666, -0.06658322,  0.68298944, -0.70186359,
       -0.68800165, -0.51711688, -0.27707907, -0.70821281, -0.06061308,
       -0.43558083,  0.52864729, -0.75267444, -0.88626458,  0.23262198,
       -0.10643358, -0.2092943 , -0.86027807, -0.00612784,  2.89

In [110]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.2s finished


In [113]:
display(y_train_pred)
display(y_train_scores)
display(y_test_pred)
display(y_test_scores)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

array([-0.94174353,  0.07290135,  0.48441584,  0.29108766, -0.05418045,
        0.4918471 , -0.6654036 , -0.74496763,  0.61358129,  2.53069441,
        0.70989566, -0.9233734 , -0.3190996 , -0.14411165, -0.04845913,
        1.48649389,  0.39497758, -0.13156485, -0.58615818,  2.47893094,
       -0.74449971,  1.07244829, -0.79663386,  0.19702066, -0.375034  ,
       -0.58571858, -0.35706745,  0.0834915 , -0.46484618, -0.90205368,
       -0.82423298,  0.63375771, -0.58896131, -0.61596971, -0.75299896,
       -0.57365835,  0.45101996, -0.67739264, -0.52267543,  1.69711294,
       -0.88246704,  2.57736525, -0.22244196, -0.20274159, -0.22125057,
        0.32045979, -0.83039248, -0.77936739,  2.84614232, -0.6348763 ,
        1.10383489,  0.00798666, -0.06658322,  0.68298944, -0.70186359,
       -0.68800165, -0.51711688, -0.27707907, -0.70821281, -0.06061308,
       -0.43558083,  0.52864729, -0.75267444, -0.88626458,  0.23262198,
       -0.10643358, -0.2092943 , -0.86027807, -0.00612784,  2.89

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

array([-0.51425619, -0.76151531, -0.42120399, -0.28567136,  1.74627361,
        0.15269175, -0.45451482,  1.31903794, -0.47873395, -0.7798278 ,
       -0.21307501, -0.939623  , -0.25232792, -0.47782434,  0.9892928 ,
       -0.68435064, -0.38644986,  0.22666652, -0.66719423, -0.89871403,
       -0.8368158 , -0.82161406, -0.55795051, -0.25825512,  0.68974193,
       -0.06380974, -0.58958001, -0.68312746,  0.25848729, -0.0530365 ,
       -0.63336392,  0.21946658,  0.88618194,  0.21017712, -0.05980769,
       -0.92908453, -0.44692676,  1.40195371,  0.71772822,  0.67322738,
       -0.49155983, -0.49921924, -0.27397379])

In [111]:
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)

print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)


On Training Data:
SUOD ROC:0.8643, precision @ rank n:0.125

On Test Data:
SUOD ROC:0.7381, precision @ rank n:0.0


In [94]:
display(X_train.head())

,var_0000,var_0001,var_0002,var_0003,var_0004,var_0005,var_0006
79,0.464410,0.335779,0.764627,0.309276,0.854649,0.087286,0.267879
161,0.643242,0.476398,0.859157,0.178578,0.641152,0.036316,0.378976
109,0.336627,0.267673,0.000000,0.398220,0.683296,0.058456,0.533381
127,0.477104,0.559034,0.000000,0.772746,0.651528,0.012492,0.336868
95,0.610833,0.464432,0.830068,0.183840,0.650862,0.028340,0.382855


In [93]:
# visualize the results
visualize(clf_name, X_train[0], y_train.[0], X_test.[0], y_test.[0], y_train_pred.[0],
          y_test_pred.[0], show_figure=True, save_figure=False)

SyntaxError: invalid syntax (<ipython-input-93-3c5f94b27622>, line 2)

In [114]:
!git clone https://github.com/chandcalnaido/ColabDev.git

Cloning into 'ColabDev'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), 260.55 KiB | 2.99 MiB/s, done.


In [120]:
%pwd
%cd ColabDev
%pwd

/content/ColabDev


'/content/ColabDev'

In [124]:
# Push to the remote repository
!git push origin main

Everything up-to-date
